# FasterRCNN

In [7]:
# For running inference on the TF-Hub module.
import tensorflow as tf

import tensorflow_hub as hub

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring the inference time.
import time

In [8]:
def download_and_resize_image(url, new_width=256, new_height=256,
                              display=False):
  _, filename = tempfile.mkstemp(suffix=".jpg")
  response = urlopen(url)
  image_data = response.read()
  image_data = BytesIO(image_data)
  pil_image = Image.open(image_data)
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.LANCZOS)
  pil_image_rgb = pil_image.convert("RGB")
  pil_image_rgb.save(filename, format="JPEG", quality=90)
  return filename

In [10]:
#module_handle = "https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1"
#detector = hub.load(module_handle)
#tf.saved_model.save(detector, "./")

INFO:tensorflow:Assets written to: ./assets


INFO:tensorflow:Assets written to: ./assets


In [28]:
# module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"

# detector = hub.load(module_handle).signatures['default']

#module_handle = "https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1"
#detector = hub.load(module_handle)
#tf.saved_model.save(detector, "./")

detector = tf.saved_model.load("./")

KeyError: 'default'

In [12]:
def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img


In [13]:
def run_detector(detector, path, min_score=0.5):
    img = load_img(path)

    # Konwertuj obraz do uint8
    converted_img = tf.image.convert_image_dtype(img, dtype=tf.uint8)[tf.newaxis, ...]

    start_time = time.time()
    result = detector(input_tensor=converted_img)
    end_time = time.time()

    result = {key: value.numpy() for key, value in result.items()}

    # Wypisanie wykrytych obiektów z uwzględnieniem progu pewności
    for score, entity in zip(result["detection_scores"][0], result["detection_classes"][0]):
        if score >= min_score:
            print(f"Object: {entity}, Score: {round(float(score), 2)}")


In [14]:
def process_fasterrcnn(image_url):
  start_time = time.time()
  image_path = download_and_resize_image(image_url, 640, 480)
  run_detector(detector, image_path)
  end_time = time.time()
  print("Inference time:",round(end_time-start_time, 2))


# Detectron2


In [15]:
# !python -m pip install pyyaml==5.1
# import sys, os, distutils.core
# # Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# # See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
# # !git clone 'https://github.com/facebookresearch/detectron2'
# # dist = distutils.core.run_setup("./detectron2/setup.py")
# # !python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
# # sys.path.insert(0, os.path.abspath('./detectron2'))

# # Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git' 

#### MOVED TO INSTALL PACKAGES SECTION

In [16]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

import requests


In [30]:
def process_detectron2(image_url):
    response = requests.get(image_url)
    if response.status_code == 200:
        image_data = np.asarray(bytearray(response.content), dtype="uint8")
        im = cv2.imdecode(image_data, cv2.IMREAD_COLOR)
        
        start_time = time.time()
        cfg = get_cfg()
        cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
        cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
        cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")

        # Ustawienie modelu do używania CPU
        cfg.MODEL.DEVICE = "cpu"

        predictor = DefaultPredictor(cfg)
        outputs = predictor(im)

        # Extract the classes of detected objects
        instances = outputs["instances"]
        classes = instances.pred_classes
        class_names = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes
        detected_objects = [class_names[i] for i in classes]
        
        end_time = time.time()
        
        print(detected_objects)
        print("Inference time:", round(end_time - start_time, 2))

# Images


In [22]:
image_urls = [
  "https://images.unsplash.com/photo-1600716051809-e997e11a5d52",
  "https://plus.unsplash.com/premium_photo-1661584150981-10626721db7a",
  "https://images.unsplash.com/photo-1716117273853-75a1989029f2"
  ]

# Results


In [31]:
for url in image_urls:
  print("Image: " + url)
  #print("Faster RCNN:")
  #process_fasterrcnn(url)
  print("Detectron 2:")
  process_detectron2(url)
  print("CLIP:")
  process_clip(url)
  print("\n")

Image: https://images.unsplash.com/photo-1600716051809-e997e11a5d52
Detectron 2:
[06/03 15:53:02 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x/138205316/model_final_a3ec72.pkl ...


model_final_a3ec72.pkl: 254MB [00:46, 5.50MB/s]                                                                                                                                                                    
/home/bihius/venv/lib/python3.11/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


['orange', 'orange', 'dining table', 'knife']
Inference time: 57.86
CLIP:
Wstawiono dane dla obrazu: photo-1600716051809-e997e11a5d52


Image: https://plus.unsplash.com/premium_photo-1661584150981-10626721db7a
Detectron 2:
[06/03 15:54:03 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x/138205316/model_final_a3ec72.pkl ...
['person', 'bottle', 'mouse', 'book']
Inference time: 10.35
CLIP:
Wstawiono dane dla obrazu: premium_photo-1661584150981-10626721db7a


Image: https://images.unsplash.com/photo-1716117273853-75a1989029f2
Detectron 2:
[06/03 15:54:19 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x/138205316/model_final_a3ec72.pkl ...
['person', 'person', 'person', 'umbrella', 'person', 'bicycle', 'chair', 'person', 'person', 'chair', 'person', 'person', 'h

# CLIP


In [27]:
import clip
import torch
from PIL import Image
import requests
import io
import psycopg2
from psycopg2 import sql
import hashlib

# Inicjalizacja Supabase
db_user = "postgres"
db_password = "your-super-secret-and-long-postgres-password"  # Wstaw swoje hasło
db_host = "10.15.10.254"
db_port = "5432"
db_name = "postgres"

try:
    conn = psycopg2.connect(
        dbname=db_name,
        user=db_user,
        password=db_password,
        host=db_host,
        port=db_port
    )
    print("Połączono z bazą danych PostgreSQL")
except Exception as e:
    print(f"Nie udało się połączyć z bazą danych: {e}")

# Funkcja do tworzenia tabeli, jeśli jeszcze nie istnieje
def create_table(conn):
    try:
        with conn.cursor() as cursor:
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS image_features (
                id SERIAL PRIMARY KEY,
                image_name TEXT NOT NULL,
                image_url TEXT NOT NULL,
                image_hash TEXT NOT NULL,
                image_feature FLOAT8[] NOT NULL
            );
            """)
            conn.commit()
            print("Tabela utworzona (jeśli jeszcze nie istniała)")
    except Exception as e:
        print(f"Nie udało się utworzyć tabeli: {e}")

# Funkcja do wstawiania danych do tabeli
def insert_image_feature(conn, image_name, image_url, image_hash, image_feature):
    try:
        with conn.cursor() as cursor:
            cursor.execute(
                """
                INSERT INTO image_features (image_name, image_url, image_hash, image_feature)
                VALUES (%s, %s, %s, %s);
                """,
                (image_name, image_url, image_hash, image_feature)
            )
            conn.commit()
            print(f"Wstawiono dane dla obrazu: {image_name}")
    except Exception as e:
        print(f"Nie udało się wstawić danych: {e}")

# Funkcja do sprawdzania, czy obraz o podanym hashu już istnieje w bazie danych
def image_exists(conn, image_hash):
    try:
        with conn.cursor() as cursor:
            cursor.execute(
                """
                SELECT EXISTS(SELECT 1 FROM image_features WHERE image_hash = %s);
                """,
                (image_hash,)
            )
            return cursor.fetchone()[0]
    except Exception as e:
        print(f"Nie udało się sprawdzić, czy obraz istnieje: {e}")
        return False

# Funkcja do generowania hasha dla obrazu
def generate_image_hash(image):
    hash_md5 = hashlib.md5()
    for chunk in iter(lambda: image.read(4096), b""):
        hash_md5.update(chunk)
    return hash_md5.hexdigest()

# Inicjalizacja modelu CLIP
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Pobieranie i przetwarzanie obrazów
def process_clip(image_url):
    try:
        response = requests.get(image_url)
        image_name = image_url.split("/")[-1]  # Wydobycie nazwy pliku z URL
        image = Image.open(io.BytesIO(response.content))

        # Generowanie hasha dla obrazu
        image_hash = generate_image_hash(io.BytesIO(response.content))

        # Sprawdzanie, czy obraz już istnieje w bazie danych
        if image_exists(conn, image_hash):
            print(f"Obraz {image_name} już istnieje w bazie danych.")
            pass

        image = preprocess(image).unsqueeze(0).to(device)

        with torch.no_grad():
            image_feature = model.encode_image(image).cpu().numpy().tolist()

        # Wstawianie wektora do bazy danych
        insert_image_feature(conn, image_name, image_url, image_hash, image_feature)
    except Exception as e:
        print(f"Nie udało się przetworzyć obrazu z URL: {image_url}, błąd: {e}")

# Tworzenie tabeli, jeśli jeszcze nie istnieje
create_table(conn)


Połączono z bazą danych PostgreSQL
Tabela utworzona (jeśli jeszcze nie istniała)


In [ ]:
import numpy as np

def search_images(query, conn, model, device, top_k=5):
    # Przetwórz zapytanie tekstowe na wektor cech za pomocą CLIP
    text = clip.tokenize([query]).to(device)

    with torch.no_grad():
        text_features = model.encode_text(text).cpu().numpy()

    # Pobierz wszystkie wektory cech obrazów z bazy danych
    try:
        with conn.cursor() as cursor:
            cursor.execute("SELECT image_url, image_feature FROM image_features;")
            rows = cursor.fetchall()
            image_urls = [row[0] for row in rows]
            image_features = np.array([row[1] for row in rows])
    except Exception as e:
        print(f"Nie udało się pobrać danych z bazy: {e}")
        return []

    # Oblicz podobieństwo między wektorem zapytania a wektorami cech obrazów
    similarities = np.dot(image_features, text_features.T).squeeze()

    # Znajdź top_k najbardziej podobnych obrazów
    top_indices = similarities.argsort()[-top_k:][::-1]
    results = [(image_urls[i], similarities[i]) for i in top_indices]

    return results

# Przykładowe użycie

pre_query = "a photo of a "
user_query = "umbrellas and tables"
final_query = str(pre_query+user_query)
results = search_images(final_query, conn, model, device)

# Wyświetlanie wyników
for img_url, score in results:
    print(f"Image URL: {img_url}, Similarity Score: {score:.4f}")


# Install packages

In [ ]:
!pip install tensorflow tensorflow_hub matplotlib requests pillow psycopg2-binary opencv-python torch

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
!pip install git+https://github.com/openai/CLIP